The line below simple requires a library called [SQLite](https://www.sqlite.org/index.html) which allows us to play around with SQL using a very simple in-memory database which doesn't require a connection to an outside server.
When you connect to a real database, you will use another driver such as [PgJDBC](https://jdbc.postgresql.org/about/about.html) for PostgreSQL.

In [1]:
%maven org.xerial:sqlite-jdbc:3.25.2

In [2]:
import java.sql.Connection;
import java.sql.DriverManager;

// First we get a connection to the database
// "sqlite" here specifies the "driver" we are using.
// Everything after the : are parameters to the driver
// (in this case, we specify an in-memory database,
//  this also means we don't actually need a username and password)

Connection connection = DriverManager.getConnection("jdbc:sqlite::memory:", "username", "password");

// Note that to use PostgreSQL, you would use something like
// jdbc:postgresql://hostname:portNumber

Now that we have a connection, we can actually execute SQL statements against the database.
First we need to create a `Statement` object which allows us execute statements.
We can then use this statement to actually execute queries and updates.
Let's start by creating a table.

In [3]:
import java.sql.Statement;

Statement statement = connection.createStatement();
statement.execute("CREATE TABLE user(id INT PRIMARY KEY, firstName VARCHAR(10))");

false

`false` here returned from `execute` means that our statement was executed successfully, but produced no results (as expected).
In general, an error could occur trying to execute our SQL, resulting in an `SQLException`.
We can ignore those in this notebook, but you'll have to handle them when you write a real program.
Let's try inserting some data into our new table.
When we insert data, we'll use a **prepared statement**.
A prepared statement allows us to efficiently insert parameters into our SQL and helps make sure our programs will not be vulnerable to [SQL injection](https://en.wikipedia.org/wiki/SQL_injection) attacks.

In [4]:
import java.sql.PreparedStatement;

PreparedStatement prepared = connection.prepareStatement("INSERT INTO user VALUES (?, ?)");

The two `?` in the statement above stand for parameters which will be replaced by the actual values we want.
The [`PreparedStatement`](https://docs.oracle.com/javase/8/docs/api/java/sql/PreparedStatement.html) class contains a number of methods starting with `set` which we use to set the parameters before execution.

In [5]:
prepared.setInt(1, 32);         // Set the first parameter (user ID) to 32
prepared.setString(2, "Neha");  // Set the second parameter (user name) to "Neha"
prepared.execute();
prepared.getUpdateCount();

1

As you can see, after setting the parameters, we just call the `execute` method which executes the `INSERT` statement.
The `getUpdateCount` call shows us that one row was updated (in this case, inserted).
Let's insert a few more rows.

In [6]:
prepared.setInt(1, 40);
prepared.setString(2, "Carlos");
prepared.execute();

prepared.setInt(1, 47);
prepared.setString(2, "Josh");
prepared.execute();

false

As you can see, we used the same `PreparedStatement` object to perform these insertions.
Again we see `false` returned by `execute` since an `INSERT` statement does not produce results.
Now let's run a simple query.
Our query has no parameters, so we'll use the `executeQuery` method on the `Statement` object we created earlier.

In [7]:
import java.sql.ResultSet;

ResultSet results = statement.executeQuery("SELECT * FROM user");

`executeQuery` returns an object of type [ResultSet](https://docs.oracle.com/javase/8/docs/api/java/sql/ResultSet.html).
A `ResultSet` has a number of `get` methods to get values of columns in our results.
(This is similar to the `set` methods on `PreparedStatement` objects).
To get all the results, we loop until the `next` until the  returns `false`, which means we've hit the last row.
Otherwise, `next` will advance our `results` object to the next row in the result.
Note that you can only go through the results **once**.
If you need to do multiple things with result, you'll either need to run the query again, or store the data somewhere else.

In [8]:
while (results.next()) {
  System.out.println(results.getInt(1) + " | " + results.getString(2));
}

32 | Neha
40 | Carlos
47 | Josh


That's it for this basic overview of SQL in Java.
You should now be able to get started writing your own programs which interact with a database using this as a starting point.
If you're just viewing the static output of this tutorial, go to [the repository](https://github.com/michaelmior/db-notebooks) and click the launch button.
This will allow you to edit all the code interactively and try modifying it as you wish.